In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_linnerud
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import joblib
import matplotlib.pyplot as plt

In [3]:
def load_and_normalize_dataset():
    DataSet = load_linnerud()
    DataFrame = pd.DataFrame(DataSet.data, columns= DataSet.feature_names)
    

    DataFrame['Weight'] = DataSet.target[:,0]
    DataFrame['Waist'] = DataSet.target[:,1]
    DataFrame['Pulse'] = DataSet.target[:,2]

    normalizer = StandardScaler()
    normalized_Input_Fields = normalizer.fit_transform (DataFrame.iloc[:,: len(DataSet.feature_names)])

    return normalized_Input_Fields, (DataFrame.iloc[:, 3:]).values


def Split_into_K_Folds(K, X,Y):

    indices = np.arange(len(X))
    np.random.shuffle(indices)
    
    X = X[indices]
    Y = Y[indices]

    size_of_folds = [len(X)//K for i in range(K)]

    for i in range(len(X)%K ):
        size_of_folds[i] += 1 
    
    S_index = 0
    E_index = 0

    Folds = []

    for i in range (K):
        S_index =  E_index
        E_index = S_index + size_of_folds[i]
        Folds.append((X[S_index:E_index],Y[S_index:E_index]))
    
    return Folds

def K_Fold_Validation (K,X,Y):

    Folds = Split_into_K_Folds(K,X,Y)
    accuracy = []

    for i in range (K):
        test_x = Folds[i][0]
        test_y = Folds[i][1]

        
        train_x = np.vstack ([Folds[j][0] for j in range(K) if j!=i]) # vstack will combine all folds X vertically
        train_y = np.vstack ([Folds[j][1] for j in range(K) if j!=i]) # vstack will combine all folds Y horizontal

        model = LinearRegression()
        model.fit(train_x, train_y)

        predict_y = model.predict(test_x)
        accur = mean_squared_error(test_y, predict_y)
        accuracy.append(accur)

        print (f'Mean Square Error: {accur:.4f}')

        if i == K-1:
            joblib.dump(model, 'physiological_measurements_model.pkl')
    print (f'Mean Mean Square Error: {np.mean(accuracy):0.4f}')

def predict_physiological_measurements (X):
    model = joblib.load('physiological_measurements_model.pkl')
    Y = model.predict (X)
    return Y


In [4]:
X,Y = load_and_normalize_dataset()
K_Fold_Validation (5, X,Y)

random_indices =[ np.random.randint(len(X)) for i in range (5)]

sample_of_5_people = X[random_indices]
result = predict_physiological_measurements(sample_of_5_people)
print ()
print ('<----------(Result)--------->')
for i,res in enumerate(result):
    
    print (f'Person {i+1} have Weight {res[0]:0.4f} KG, Waist {res[1]:0.4f} CM, Heart Rate {res[1]:0.4f} BPM.')
   
    



Mean Square Error: 164.6609
Mean Square Error: 205.3451
Mean Square Error: 519.6874
Mean Square Error: 442.2392
Mean Square Error: 120.6801
Mean Mean Square Error: 290.5225

<----------(Result)--------->
Person 1 have Weight 191.3477 KG, Waist 37.4216 CM, Heart Rate 37.4216 BPM.
Person 2 have Weight 196.6295 KG, Waist 38.0666 CM, Heart Rate 38.0666 BPM.
Person 3 have Weight 155.4923 KG, Waist 31.7118 CM, Heart Rate 31.7118 BPM.
Person 4 have Weight 183.2558 KG, Waist 35.9008 CM, Heart Rate 35.9008 BPM.
Person 5 have Weight 191.3477 KG, Waist 37.4216 CM, Heart Rate 37.4216 BPM.
